folfiri_dynamics: *script to obtain data for running ParseQSP.m and PlotQSP.m *

Author: Aparajita Budithi

 (c) Shahriyari Lab https://sites.google.com/site/leilishahriyari/
 
 part of https://github.com/ShahriyariLab/Data-driven-mathematical-model-of-FOLFIRI-treatment-for-colon-cancer
 
 If using this or related code please cite
   
   Budithi, A.; Su, S.; Kirshtein,A.; Shahriyari L.\
     Data driven mathematical model of FOLFIRI treatment for colon cancer. //Cancers.\
     (Manuscript submitted for publication).

In [4]:

from qspmodel import *
from folfiri_qspmodel import *
import pandas as pd
import csv
import os
import scipy as sp

if not os.path.exists('Results/'):
    os.makedirs('Results/All_Dynamic/')
else:
    if not os.path.exists('Results/All_Dynamic/'):
        os.makedirs('Results/All_Dynamic/')


# some global parameters
clusters=5 #number of clusters

#number_of_cycles=6
cycle_time=14
injection_time=2
number_of_cycles=12

start_treatment = 0
T=2000
t=np.linspace(0, T, 20001)

# pre-defined parameters for treatment deltas, cell_steady, cell_extreme, drug_ratio, drug_eff, 5FU_killrate
drug_deltas=[71.3, 1.45, 2.56]
cell_steady_ind=[5, 6, 13]
cell_extreme_ind=[5, 6, 12]
drug_levels=np.array([770, 300, 725])/cycle_time #per day; median drug dosages from patient data
drug_min=np.array([598, 208, 75])/cycle_time #per day; min drug dosages from patient data
drug_ratio=drug_levels/drug_min
drug_eff=[0.2, 0.4, 0.05]
FU_killrate=100

# create parameters for step function 
infusion_indices=np.concatenate([15*np.ones(number_of_cycles), 16*np.ones(number_of_cycles), 17*np.ones(number_of_cycles)]).astype(int)
FUinfusion=np.array([2/24, 2])
IRinfusion=np.array([0, 1/24])
LVinfusion=np.array([1/24, 2/24])
infusion_intervals=np.concatenate([[start_treatment+n*cycle_time+FUinfusion for n in range(number_of_cycles)],
                                    [start_treatment+n*cycle_time+IRinfusion for n in range(number_of_cycles)],
                                    [start_treatment+n*cycle_time+LVinfusion for n in range(number_of_cycles)]])
infusion_values=np.concatenate([(cycle_time/(FUinfusion[1]-FUinfusion[0]))*drug_deltas[0]*np.ones(number_of_cycles),
                                (cycle_time/(IRinfusion[1]-IRinfusion[0]))*drug_deltas[1]*np.ones(number_of_cycles),
                                (cycle_time/(LVinfusion[1]-LVinfusion[0]))*drug_deltas[2]*np.ones(number_of_cycles)])

#other parameters
nvar=Colon_5fu_Functions().nvar # number of variables
nparam=Colon_5fu_Functions().nparam # number of parameters

# infusion function
r= step_vector(nvar, indices=infusion_indices, intervals=infusion_intervals, values=infusion_values)


# define cell data for non-treatment parameter derivation
# rates acquired from bio research [delTn delTh delTc delTr delD delM lamgCmax lamgCmin delmu1 delmu2 delmu3 delH delIg delGb]
globvals=np.array([9.4951e-4, 0.231, 0.406, 0.231, 0.277, 0.02, 7.5e-3, 6.7152e-04, 1.07, 4.62, 58.7, 33.27, 499])
# average values of each variable across all patients [Tc mu1 Ig Gb]
meanvals=np.array([2920.305826, 132.316278, 6.603464, 20017.990343])
extremevals=np.array([2.67309712e+04, 1.23113649e+04, 1.91073391e+04, 7.21023854e+03,
                              3.41730382e+03, 4.32746936e+03, 2.31601361e+04, 3.24717521e+05,
                              1.62358760e+05, 1.05768591e+03, 1.39828734e+04, 2.46968870e+04,
                              1.02214368e+02, 1.63407223e+05])
celldata=[globvals, extremevals, meanvals]


clustercells=pd.read_csv('input/Large_Tumor_cell_data.csv').to_numpy()

#Computations of dynamics
print('Starting dynamics computations')

#initial conditions at steady-state
IC= np.ones((clusters,nvar))
IC[:,-3:]=0;

#initial conditions from data
#IC=np.array(pd.read_csv('input/New_Initial_Conditions.csv'))
#IC=np.block([IC,np.zeros((clusters,3))])

for cluster in range(clusters):
    print('Starting computations for cluster '+str(cluster+1)+' of 5') 
    filename='V63-cluster-'+str(cluster+1)+'-of-5-results-'

    QSP0=QSP.from_data([clustercells[cluster]]+celldata)
    qspcore=Colon_5fu_Functions(parameters=QSP0.par)
    QSP_=QSP.from_data(([drug_deltas, (clustercells[cluster,cell_steady_ind]/extremevals[cell_extreme_ind]), drug_ratio, drug_eff, FU_killrate]), qspcore=qspcore)
    
    print(' Parameters set. Computing the solution')
    u, _ = QSP_.solve_ode(t, IC[cluster], 'given', inhomogeneity=r, jumps=True)
    
    wr=np.empty((sol.t.size, nvar+1))
    wr[:,0]=sol.t
    wr[:,1:]=u
    #scaling for drugs: 
    for i in range(1,4):
        wr[:,-i]= wr[:,-i]*(drug_levels[-i]/drug_deltas[-i])
    c=csv.writer(open('Results/All_Dynamic/'+filename+'dat.csv',"w"))  
    c.writerow(['time']+QSP_.variable_names())
    c.writerows(wr)
    del c

print('end.')


Starting dynamics computations
Starting computations for cluster 1 of 5
 Parameters set. Computing the solution
Starting computations for cluster 2 of 5
 Parameters set. Computing the solution
Starting computations for cluster 3 of 5
 Parameters set. Computing the solution
Starting computations for cluster 4 of 5
 Parameters set. Computing the solution
Starting computations for cluster 5 of 5
 Parameters set. Computing the solution
end.
